ação.
API: https://www.sptrans.com.br/desenvolvedores
ENDPOINTS:https://www.sptrans.com.br/desenvolvedores/api-do-olho-vivo-guia-de-r
eferencia/documentacao-api/
Utilizar o GTFS da SPTRANS para dados complementares (Dados
Estáticos/Cadastrais).
Enriquecer os dados de paradas com o endereço da localidade, através de latitude e
longitude.
Dados do GTFS: https://gtfs.org/documentation/schedule/reference/


In [ ]:
import geopy.distance



In [ ]:
%%timeit
coords_1 = (52.2296756, 21.0122287)# (lat, lon)
coords_2 = (52.406374, 16.9251681) # (lat, lon)

geopy.distance.geodesic(coords_1, coords_2).km

# The latitude of Sao Paulo City, Brazil is -23.533773, and the longitude is -46.625290.

In [ ]:
import collections

import datetime
previsao = result.json()

def convert_time(text):
    h,m = text.split(':')
    return datetime.datetime(2025, 1, 17, int(h), int(m), 0)

hora_atual = convert_time(previsao.get('hr'))
print(hora_atual)

onibus = collections.defaultdict(list)
for ps in previsao.get('ps',[]):
    cod_parada = ps.get('cp','?')
    ps_px = ps.get('px') # long
    ps_py = ps.get('py') # lat 
    vs = ps.get('vs',[])
    for veiculo in vs:
        vs_p = veiculo.get('p')
        vs_t = veiculo.get('t')
        vs_px = veiculo.get('px')
        vs_py = veiculo.get('py')
        print(vs_p)
        onibus[vs_p].append((cod_parada, vs_t, round(geopy.distance.geodesic((vs_py, vs_px), (ps_py, ps_px)).km,ndigits=3)))
        
    

onibus    

In [ ]:
import datetime


for cod,paradas in onibus.items():
    print(cod)
    paradas = sorted(paradas, key=lambda x: int(x[1].replace(':','')))
    p0 = paradas[0]
    for p in paradas:
        hora_parada = convert_time(p[1])
        diff_hora = (hora_parada -  hora_atual).seconds
        speed = round(p[2] / (diff_hora/3600), 3)
        print[p[0], para, diff_hora, speed] )


In [ ]:
import openlayer


In [ ]:
# !pip install ipyleaflet jupyter-leaflet
# conda install conda-forge::nodejs
# !jupyter labextension install @jupyter-widgets/jupyterlab-manager jupyter-leaflet
import ipyleaflet

In [ ]:
from ipyleaflet import Map, Marker

center = (-23.5558, -46.6396)

m = Map(center=center, zoom=10)
marker = Marker(location=center, draggable=True)
m.add(marker);

In [ ]:


result = posicao_por_linha(session, 1171).json()
# print(json.dumps(result.json(), indent=2))
vs = result.get('vs', [])
for veiculo in vs:
    print(veiculo)
    px = veiculo.get('px')
    py = veiculo.get('py')
    print(py,px)
    marker = Marker(location=(py,px), draggable=True)
    m.add(marker);

display(m)

In [ ]:
import geopy.distance

coords_1 = (52.2296756, 21.0122287)
coords_2 = (52.406374, 16.9251681)

print(geopy.distance.geodesic(coords_1, coords_2).km)

In [ ]:
import pandas as pd

BASE_PATH = 'trafi_gtfs'

df_agency = pd.read_csv(f'{BASE_PATH}/agency.txt', header=0)
df_agency.head()

In [ ]:
df_calendar = pd.read_csv(f'{BASE_PATH}/calendar.txt', header=0)
df_calendar.head()

In [ ]:
df_fare_attributes = pd.read_csv(f'{BASE_PATH}/fare_attributes.txt', header=0)
df_fare_attributes.head()

In [ ]:
df_fare_rules = pd.read_csv(f'{BASE_PATH}/fare_rules.txt', header=0)
print(df_fare_rules.shape)
df_fare_rules.head()


In [ ]:
df_frequencies = pd.read_csv(f'{BASE_PATH}/frequencies.txt', header=0)
print(df_frequencies.shape)
df_frequencies.head()

In [ ]:
df_routes = pd.read_csv(f'{BASE_PATH}/routes.txt', header=0)
print(df_routes.shape)

df_routes.head()

# shapes

Primary key (shape_id, shape_pt_sequence)

Shapes describe the path that a vehicle travels along a route alignment, and are defined in the file shapes.txt. Shapes are associated with Trips, and consist of a sequence of points through which the vehicle passes in order. Shapes do not need to intercept the location of Stops exactly, but all Stops on a trip should lie within a small distance of the shape for that trip, i.e. close to straight line segments connecting the shape points. The shapes.txt file should be included for all route-based services (not for zone-based demand-responsive services).

In [ ]:
df_shapes = pd.read_csv(f'{BASE_PATH}/shapes.txt', header=0)
df_shapes.describe()

In [ ]:
from ipyleaflet import Map, Marker, FullScreenControl

shape_pos = df_shapes[df_shapes['shape_id'] == 81072][['shape_pt_lat', 'shape_pt_lon']].values 

avg_lat = sum(x[0] for x in shape_pos)/len(shape_pos)
avg_long = sum(x[1] for x in shape_pos)/len(shape_pos)

control = FullScreenControl()

# center = (-23.5558, -46.6396)
center = (avg_lat, avg_long)


m1 = Map(center=center, zoom=12)
m1.add_control(control)

for p in shape_pos:
    marker = Marker(location=tuple(p), draggable=False)
    m1.add(marker);

display(m1)

In [ ]:
df_stop_times = pd.read_csv(f'{BASE_PATH}/stop_times.txt', header=0)
df_stop_times.head()

In [ ]:
df_stops = pd.read_csv(f'{BASE_PATH}/stops.txt', header=0)
print(df_stops.shape)
df_stops.head()


In [ ]:
from ipyleaflet import Map, Marker, FullScreenControl
shape_pos = df_stops[['stop_lat', 'stop_lon']].values[:500] 

avg_lat = sum(x[0] for x in shape_pos)/len(shape_pos)
avg_long = sum(x[1] for x in shape_pos)/len(shape_pos)

control = FullScreenControl()

# center = (-23.5558, -46.6396)
center = (avg_lat, avg_long)


m1 = Map(center=center, zoom=12)
m1.add_control(control)

for p in shape_pos:
    marker = Marker(location=tuple(p), draggable=False)
    m1.add(marker);

display(m1)

In [ ]:
# https://gtfs.org/documentation/schedule/reference/#tripstxt

df_trips = pd.read_csv(f'{BASE_PATH}/trips.txt', header=0)
df_trips.head()

In [ ]:
trips (route_id -> Shape_id)
df_routes (-)
stops (-)


In [ ]:
df_stops

In [ ]:
df_shapes


In [ ]:
def get_shape_from_id(shape_id):
    return df_shapes[df_shapes['shape_id'] == shape_id][['shape_pt_lat', 'shape_pt_lon', 'shape_pt_sequence']].values

get_shape_from_id(62557)

In [ ]:
def get_trips_stop_seq(trip_id):
    trip_seq = df_stop_times[df_stop_times['trip_id'] == trip_id][['stop_id','stop_sequence']].values
    result = [ (i,tuple(df_stops[df_stops['stop_id'] == s].values[0])) for s,i in trip_seq ]
    return result
get_trips_stop_seq('1012-10-0')

In [ ]:
trip_seq = get_trips_stop_seq('1012-10-0')
trip_seq

In [ ]:
df_trips[df_trips['route_id'] == '1012-10']

In [ ]:
df_routes[df_routes['route_id'] == '1012-10']

In [ ]:
route_id = '1012-10'

route_data = df_routes[df_routes['route_id'] == route_id]
print(f'Rota ID = {route_id}')
display(route_data)
trips = df_trips[df_trips['route_id'] == route_id]
print('Contem as trips:')
display(trips)

In [ ]:
trip_seq = get_trips_stop_seq('1012-10-0')
trip_seq

In [ ]:
# get_shape_from_id(81073)


In [ ]:
list_pos_exemplo = get_shape_from_id(81072)
lat_exemplo = -23.4285
long_exemplo = -46.795379

In [ ]:
%%timeit
def find_closest(list_pos, s_lat, s_long):
    # print(s_lat, s_long)
    best_pos_index = None
    best_dist = 999999
    for lat, long, shape_pt_sequence in list_pos:
        # print(lat, long, shape_pt_sequence)
        dist = geopy.distance.geodesic((s_lat, s_long), (lat, long)).meters
        if dist < best_dist:
            best_dist = dist
            best_pos_index = shape_pt_sequence
    return best_pos_index, best_dist

    

find_closest(list_pos_exemplo, lat_exemplo, long_exemplo)


In [ ]:
list_pos_exemplo[83]

In [ ]:
%time
def conciliar_trip_stop(trip_id, shape_id):
    trip_seq = get_trips_stop_seq(trip_id)
    shape = get_shape_from_id(shape_id)
    index = {}
    for i, stop in trip_seq:
        closest = find_closest(shape, stop[3], stop[4])
        # print(closest) # shape_pt_sequence, distance
        index[closest[0]] = (stop[0], closest[1])

    shape_for_trip = df_shapes[df_shapes['shape_id'] == shape_id].copy() 
    shape_for_trip['stop_id'] = [index.get(x, (None, None))[0] for x in shape_for_trip['shape_pt_sequence'].values]
    shape_for_trip['stop_distance'] = [index.get(x, (None, None))[1] for x in shape_for_trip['shape_pt_sequence'].values]
    return shape_for_trip
                     
conciliar_trip_stop('1012-10-0', 81072).head(50)

In [ ]:
d = df_stops[df_stops['stop_id'] == 301790]
lat = d.values[0][3]
long = d.values[0][4]
print(lat, long)
display(d)
geopy.distance.geodesic((-23.431827, -46.787153), (lat, long)).meters

## Ideias:

* Tempo de atraso nos pontos (stops)
* Velocidade media
* Deteccao de eventos/emergencias


## Tasks: 

- Subir MiniIO
- manter um sistema gravando no MiniOI dados de posicao
- sistema para processar os dados - Pyspark

# docker composer
- MiniIO
- PySpark
- Trino
- Metabase / SuperSet
- 


## Questoes
- Todas as posicoes? Ou só algumas linhas?
- frequencia de atualizacao (1 minuto)


previsão da linha
-> 


acertividade da previsão
-> differenca entre o previsto e o realizado



-> pega a posicao dos onibus usando a API (horario de referencia, prefixo veiculo, linha, etc)


-> shapes -> unir shapes com stops (percuso + paradas)
